# ST5225 Statistical Analysis of Networks — Deep Dive
# Week 2 — Basic Quantities and Properties of Networks

## This session's network: Instagram
This is an Instagram social network data for Influence Maximization (IM) task. It was collected from Instagram on April to May 2020, from the followers of 24 private universities in Malaysia, using Instagram API and various third-party Instagram websites. It consists of mostly Malaysian users, with 70,409 nodes/users, 1,007,107 edges/connections (followees and followers).
Download "Network for IC LT.txt" from [Kaggle](https://www.kaggle.com/datasets/krpurba/im-instagram-70k). You need to register to download the file.
The CSV file contains three values in each row: Source ("follower"), Target ("followee"), and a weight, which we can ignore. 

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

# Load the file into a dataframe
df = pd.read_csv('Network for IC LT.txt', sep=' ', skiprows=1, header=None)

# Display the dataframe
print(df)

In [ ]:
# Create a directed graph from the dataframe
G = nx.from_pandas_edgelist(df, source=0, target=1, create_using=nx.DiGraph())

# Print the graph
print(G)

In [ ]:
import numpy as np

# Extract the top nodes with the highest in-degrees
top_nodes = sorted(G.in_degree(), key=lambda x: x[1], reverse=True)[:50]

# Create a subgraph with the top nodes
subgraph = G.subgraph([node for node, _ in top_nodes])

# Use spring layout
pos = nx.kamada_kawai_layout(subgraph)

# Calculate node sizes based on the square root of the in-degree
node_sizes = [np.sqrt(subgraph.in_degree(node)) * 100 for node in subgraph]  # Adjust the multiplier as needed for visualization

# Visualize the subgraph with node sizes proportional to the square root of the in-degree
nx.draw(subgraph, pos, with_labels=False, node_size=node_sizes)
plt.show()

## Degrees

We first calculate the degree distribtion of this directed graph. We can calculate three types of degrees: in-degree, out-degree, and total degree. The in-degree of a node is the number of edges coming into the node, the out-degree of a node is the number of edges going out of the node, and the total degree of a node is the sum of the in-degree and out-degree of the node.

For this dataset, we will focus on the in-degrees ("number of followers").

In [ ]:
degrees = G.in_degree()
print(degrees)


Let's sort this data.

In [ ]:
degree_seq = sorted((d for n, d in degrees), reverse=True)
print(degree_seq[:20])
print(degree_seq[-20:])

We now calculate the degree distribution.

In [ ]:
from collections import Counter

# Create a Counter object for the in_degree_seq list
degree_counter = Counter(degree_seq)

degree_dist = pd.DataFrame.from_dict(degree_counter, orient='index', columns=['Frequency'])
degree_dist.index.name = 'Degree'
degree_dist.reset_index(inplace=True)
degree_dist = degree_dist[degree_dist['Degree'] != 0]

print(degree_dist)

Let's plot this distribution.

In [ ]:
plt.scatter(degree_dist['Degree'], degree_dist['Frequency'], s=2)
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.title('Degree Distribution')
plt.show()

This is not very useful. Let's plot this on the log-log scale.

In [ ]:
plt.loglog(degree_dist['Degree'], degree_dist['Frequency'], marker='o', linestyle='None', markersize=2)
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.title('Degree Distribution (Log-Log Scale)')
plt.ylim([1, 10000])  # Set the y-axis range from 1 to 1000
plt.show()

In [ ]:
import numpy as np

# Filter out the data points with degree less than 10 
high_degree_dist = degree_dist[degree_dist['Degree'] >= 10]

# Convert the degree and frequency data to numpy arrays
degree = np.array(high_degree_dist['Degree'])
frequency = np.array(high_degree_dist['Frequency'])

# Take the logarithm of the degree and frequency arrays
log_degree = np.log(degree)
log_frequency = np.log(frequency)

# Perform the linear regression using numpy's polyfit function
slope, intercept = np.polyfit(log_degree, log_frequency, 1)

# Print the equation of the linear regression line
print(f"Log(Frequency) = {intercept:.4f} + {slope:.4f} * Log(Degree)")


In [ ]:
plt.loglog(degree_dist['Degree'], degree_dist['Frequency'], marker='o', linestyle='None', markersize=2)
plt.plot(degree_dist['Degree'], np.exp(intercept) * degree_dist['Degree'] ** slope, color='red', label='Fit Line')  


plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.title('Degree Distribution (Log-Log Scale)')
plt.ylim([1, 10000])  # Set the y-axis range from 1 to 1000
plt.show()

Many network scientists (especially those from the physics community) get excited when they see a straight line on a log-log plot of the degree distribution, as this implies a "power law distribution": $$d(k) = k^{-\alpha},$$ where $d(k)$ is the number of nodes of degree k and $\alpha>1$. We will encouter this phenomenon when studying preferential attachement random graphs (the "rich-get-richer phenomenon").

 However, there has been much debate recently about when a straight line is really a straight line, since on a log-log plot, many distributions look like a straight line. Our Instragram data seems not to exhibit a straight line, which is sort of surprising, since social network data is exactly the kind of data where one would expect to see a powerlaw. 

## Edge density

We now calculate the edge density. For undirected graph, the edge density is defined to be $$\frac{2 e}{n(n-1)},$$ where $e$ is the number of undirected edges and $n$ the number of vertices. For directed graphs, the edge density is defined to be $$\frac{e}{n(n-1)},$$ where $e$ is the number of directed edges. 

In [ ]:
print("Edge Density:", nx.density(G))

Is this sparse or dense? A better indicator is the average degree:

In [ ]:
# Assuming your directed graph is stored in the variable G
in_degrees = dict(G.in_degree())
out_degrees = dict(G.out_degree())

# Calculate average in-degree
avg_in_degree = sum(in_degrees.values()) / len(in_degrees)

# Calculate average out-degree
avg_out_degree = sum(out_degrees.values()) / len(out_degrees)

print("Average In-Degree:", avg_in_degree)
print("Average Out-Degree:", avg_out_degree)


The average in- and out-degrees must equal each other (why?). The value above is moderate, so one could argue this is a sparse graph. 

## Betweeness centrality

Let us now calculate the betweeness centrality of the vertices in the Instagramm network. Recall that the betweenness centrality of a vertex $v$ is calculated as the sum of the fractions of shortest paths between all pairs of vertices that pass through $v$. This value is actually computationally expensive, so we resort to a "Monte Carlo" approach, which means, we take random samples and estimate the values. Since $G$ is a directed graph, the paths considered are the directed paths only.

Let's continue with only those edges with large in-degree ("influencers").

In [ ]:
G_hi = nx.Graph(G.subgraph([node for node, degree in G.degree() if degree >= 100]))

print(G_hi)

# Calculate the betweenness centrality of the nodes in the graph; k is the number of samples to use, since calculating the true values is computationally expensive
betweenness_centrality = nx.betweenness_centrality(G_hi, k=300)


In [ ]:
num_top_vertices = 50

top_vertices_b = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:num_top_vertices]

for vertex in top_vertices_b[:10]:
  print(f"Vertex: {vertex[0]}, Centrality: {vertex[1]}")

# Create a subgraph with the top vertices
subgraph_b = nx.DiGraph(G_hi.subgraph([vertex[0] for vertex in top_vertices_b]))
subgraph_b.remove_nodes_from(list(nx.isolates(subgraph)))


In [ ]:
# Use spring layout
pos = nx.kamada_kawai_layout(subgraph_b)

# Calculate node sizes based on the square root of the in-degree
node_sizes = [np.sqrt(subgraph_b.in_degree(node)) * 100 for node in subgraph_b]  # Adjust the multiplier as needed for visualization

# Visualize the subgraph with node sizes proportional to the square root of the in-degree
nx.draw(subgraph_b, pos, with_labels=False, node_size=node_sizes)
plt.show()

## Closeness centrality

Let us now calculate the closeness centrality of the vertices in the Instagram network. Recall that the closeness centrality of a vertex $v$ is calculated as the reciprocal of the average shortest path length from $v$ to all other vertices. These values can be computed using *Dijkstra's algorithm*; see e.g. [Computing Classic Closeness Centrality, at Scale](https://www.microsoft.com/en-us/research/wp-content/uploads/2014/08/sabidussi_TR.pdf). The algorithm is exact, hence takes a few minutes to complete. It scales roughly like $\Theta(n^2)$, where $n$ is number of vertices.

In [ ]:
# Calculate the betweenness centrality of the nodes in the graph; k is the number of samples to use, since calculating the true values is computationally expensive
closeness_centrality = nx.closeness_centrality(G_hi)

In [ ]:
num_top_vertices = 50

top_vertices_c = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:num_top_vertices]

for vertex in top_vertices_c[:10]:
  print(f"Vertex: {vertex[0]}, Centrality: {vertex[1]}")

# Create a subgraph with the top vertices
subgraph_c = nx.DiGraph(G.subgraph([vertex[0] for vertex in top_vertices_c]))
subgraph_c.remove_nodes_from(list(nx.isolates(subgraph_c)))


In [ ]:
# Use spring layout
pos = nx.kamada_kawai_layout(subgraph_c)

# Calculate node sizes based on the square root of the in-degree
node_sizes = [np.sqrt(subgraph_c.in_degree(node)) * 100 for node in subgraph_c]  # Adjust the multiplier as needed for visualization

# Visualize the subgraph with node sizes proportional to the square root of the in-degree
nx.draw(subgraph_c, pos, with_labels=False, node_size=node_sizes)
plt.show()

## Eigenvector centrality

Let us now calculate the eigenvector centrality of the vertices in the Instagram network. Recall that the eigenvector centrality of a vertex $v$ is calculated by means of the adjacency matrix. Such methods are also called "spectral methods".

In [ ]:
# Calculate the eigenvalue centrality of the nodes in the graph
eigenvalue_centrality = nx.eigenvector_centrality(G_hi)

In [ ]:
num_top_vertices = 50

top_vertices_e = sorted(eigenvalue_centrality.items(), key=lambda x: x[1], reverse=True)[:num_top_vertices]

for vertex in top_vertices_e[:10]:
  print(f"Vertex: {vertex[0]}, Centrality: {vertex[1]}")

# Create a subgraph with the top vertices
subgraph_e = nx.DiGraph(G.subgraph([vertex[0] for vertex in top_vertices_e]))
subgraph_e.remove_nodes_from(list(nx.isolates(subgraph_e)))


In [ ]:
# Use spring layout
pos = nx.kamada_kawai_layout(subgraph_e)

# Calculate node sizes based on the square root of the in-degree
node_sizes = [np.sqrt(subgraph_e.in_degree(node)) * 100 for node in subgraph_e]  # Adjust the multiplier as needed for visualization

# Visualize the subgraph with node sizes proportional to the square root of the in-degree
nx.draw(subgraph_e, pos, with_labels=False, node_size=node_sizes)
plt.show()

## Triangles and Cluster Coefficient

We now calculate the triangles in the graph. To simplify things, we convert the graph into an undirected graph. 

In [ ]:
G = G.to_undirected()

In [ ]:
# Assuming G is your graph
triangles = nx.triangles(G)

# Calculate the total number of triangles
num_triangles = sum(triangles.values()) // 3

print("Number of triangles in G:", num_triangles)

Let us now calculate the number of 2-paths

In [ ]:
# Calculate the number of two-paths using degree information
num_two_paths = sum([degree * (degree - 1) // 2 for node, degree in G.degree()])
print("Number of two-paths in G:", num_two_paths)


We can now calculate the cluster coefficient as in our lecture: $$C = \frac{3\times\text{number of triangles}}{\text{number of 2-stars}}$$

In [ ]:
# Calculate the clustering coefficient
cluster_coefficient = 3 * num_triangles / num_two_paths

print(f"Cluster Coefficient: {cluster_coefficient}")


In NetworkX, this can be calculated using the nx.transitivity function


In [ ]:
print("Transitivity:", nx.transitivity(G))

It is also possible to calculate the cluster coefficient for each vertex individually: $$C_v = \frac{3\times\text{number of triangles going through $v$}}{\text{number of 2-stars with $v$ as centre}}.$$ This leads to the average cluster coefficient.

In [ ]:
cluster_coefficient = nx.average_clustering(G)
print("Average Cluster Coefficient:", cluster_coefficient)

Are these coefficients larger than one would expect? Need a null-model against which to test.

In [ ]:
# Rewire the graph G
degrees = [degree for node, degree in G.degree()]
rewired_G = nx.configuration_model(degrees)

# Convert the rewired graph to a simple graph
rewired_G = nx.Graph(rewired_G)

# Assuming G is your graph
num_triangles = sum(nx.triangles(rewired_G).values()) // 3
num_two_paths = sum([degree * (degree - 1) // 2 for node, degree in rewired_G.degree()])

# Calculate the clustering coefficient
cluster_coefficient = 3 * num_triangles / num_two_paths

print(f"Cluster Coefficient: {cluster_coefficient}")


Going by the simulation, the cluster coefficients of $G$ is larger than one would expect in a "randomly chosen graph with the same degree sequence" (configuration random graph model). That is, there are more triangles than one would expect. This is typical for social networks.  